In [1]:
from pydantic import BaseModel, Field
import nest_asyncio
nest_asyncio.apply()

from voice_agent_flow.llms import create_pydantic_azure_openai
from voice_agent_flow.agents.agent_node import AgentNode, HangUpNode
from voice_agent_flow.agents import MultiAgentRunner
from voice_agent_flow.agents import pmsg


class PoliceCallBasicInfo(BaseModel):
    """Information collected during a police call. Need to collect all the inforamation before createing this object."""
    case_location: str = Field(..., description='The location of the case')
    case_type: str = Field(..., description='The type of the case')
    description: str = Field(..., description='The description of the case')
    caller_name: str = Field(..., description='The name of the caller')
    
    
    def transfer(self) -> str:
        print("Transfer to safety_suggestion")
        return 'safety_suggestion'
    
    
class SafetySuggestionProvided(BaseModel):
    """Base on collected information, provide safety suggestion to the caller. After the caller responded to your suggestion, create this object."""
    suggestion_provided:bool = Field(..., description='Whether safety suggestion is provided to the caller')
    
    
    def transfer(self) -> str:
        print("Transfer to hangup")
        return 'hangup' 


In [2]:
model = create_pydantic_azure_openai('gpt-4o-mini')

instruction = """
You are a police call center agent(working at 110). You task is to talk with caller via telephone to collection information.
You resopnse should be berief and direct to the point.
"""


def location_reachable_on_map(location: str) -> bool:
    """Search if the location is reachable on map."""
    print(f"Location Checking Result, location: {location}, reachable: True")
    return True

agents = {
    "police_call_basic_info": AgentNode(
        name="police_call_basic_info",
        model=model,
        instruction=instruction,
        task_cls= PoliceCallBasicInfo,
        step_instruction=(
            "Collect basic information about the police call including case location, case type, description and caller name.) "
            "ask the question one at a time, do not ask multiple questions in one message."
            "When caller told you the location, make sure to call the tool to check if the location is reachable on map."
            "If the location is not reachable, ask the caller to provide more details about the location."
        ),
        examples=["请问您遇到什么紧急情况？ / 发生在哪里？/ 能简单描述一下吗？/ 您的姓名是？"],
        tools = [location_reachable_on_map],
        ),
    
    "safety_suggestion": AgentNode(
        name="safety_suggestion",
        model=model,
        instruction=instruction,
        task_cls= SafetySuggestionProvided,
        step_instruction="Based on the collected information, provide safety suggestion(A clear command to keep safe) to the caller. After the caller responded to your suggestion, create the schema.",
        examples=["请您保持冷静，.......(provide clear safety suggestion)", "请您打开窗户，保持空气流通"],
        ),
    
    "hangup": HangUpNode(
        model = model)
    }
      
      
runner = MultiAgentRunner(
    agents = agents, 
    entry_agent_name="police_call_basic_info", 
    ending_message="好的，我们这就派人过去处理您的情况，请您保持电话畅通！"
) 
 

In [3]:
from voice_agent_flow.agents.events import (
    AgentTextStream,
    ToolCallsOutput,
    ToolCallResult,
    StructuredOutput,
    AgentHandoff,
    HangupSignal
)

In [4]:
memory = [
    
]

async def _run(query:str = None):
    print(f"🤖[{runner.current_agent.name}]...Working.")
    if query is not None:
        msg = pmsg.user(query)
        memory.append(msg)
    
    rerun = False
    output_text = ""
    async for event in runner.run(message_history = memory):
        
        if isinstance(event.event, AgentTextStream):
            output_text += event.event.delta
            
        if isinstance(event.event, ToolCallsOutput):
            memory.append(pmsg.tool_call(
                tool_name = event.event.message['tool_name'],
                args = event.event.message['args'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, ToolCallResult):
            memory.append(pmsg.tool_return(
                tool_name = event.event.message['tool_name'],
                content = event.event.message['content'],
                tool_call_id=event.event.message['tool_call_id']
            ))
            
        if isinstance(event.event, AgentHandoff):
            print(event.event)
            rerun = True
            
        if isinstance(event.event, HangupSignal):
            print(event.event)
            print("Conversation Ended with Hangup Signal.")
            
    if len(output_text) > 0:
        memory.append(pmsg.assistant(output_text))
        print(output_text)
        
    return rerun
        
async def run(query:str = None):
    rerun = await _run(query)
    if rerun:
        await _run()
    
        

In [5]:
await run("你好，我家进水了")

🤖[police_call_basic_info]...Working.
请问发生在哪里？


In [6]:
await run("朝阳区，麦子店32号")

🤖[police_call_basic_info]...Working.
Location Checking Result, location: 朝阳区，麦子店32号, reachable: True
请问这是哪种类型的案件？比如火灾、盗窃等？


In [7]:
await run("房屋受损，危险")

🤖[police_call_basic_info]...Working.
能简单描述一下具体情况吗？


In [8]:
await run("阀门坏了，厕所一直漏水")

🤖[police_call_basic_info]...Working.
您的姓名是？


In [9]:
await run("王小明")

🤖[police_call_basic_info]...Working.
Transfer to safety_suggestion
AgentHandoff(status=None, message={'source_agent_name': 'police_call_basic_info', 'target_agent_name': 'safety_suggestion'})
🤖[safety_suggestion]...Working.
请您立即关闭水源阀门以防止进一步漏水，同时确保不要直接接触漏水的地方，保持安全。


In [10]:
await run("嗯")

🤖[safety_suggestion]...Working.
Transfer to hangup
AgentHandoff(status=None, message={'source_agent_name': 'safety_suggestion', 'target_agent_name': 'hangup'})
🤖[hangup]...Working.
感谢您的配合。请注意安全，如果您有其他问题或需要进一步的帮助，请随时告知。祝您生活愉快，再见。


In [11]:
await run("嗯")

🤖[hangup]...Working.
HangupSignal(status=None, message=DoHangUp())
Conversation Ended with Hangup Signal.


In [12]:
memory

[ModelRequest(parts=[UserPromptPart(content='你好，我家进水了', timestamp=datetime.datetime(2026, 2, 24, 8, 52, 23, 28822, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='请问发生在哪里？')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 24, 8, 52, 25, 132861, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[UserPromptPart(content='朝阳区，麦子店32号', timestamp=datetime.datetime(2026, 2, 24, 8, 52, 25, 143301, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ToolCallPart(tool_name='location_reachable_on_map', args='{"location":"朝阳区，麦子店32号"}', tool_call_id='call_xJqhxdBMdQz6VDVCLEFS9S53')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 24, 8, 52, 26, 762607, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[ToolReturnPart(tool_name='location_reachable_on_map', content=True, tool_call_id='call_xJqhxdBMdQz6VDVCLEFS9S53', timestamp=datetime.datetime(2026, 2, 24, 8, 52, 26, 765692, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(co